In [ ]:
# This notebook can be used to replicate our results from the unfolding appendix.
# Prerequisites: Full set of unfolded results using NumpyToysToUnfoldedMeasurements.ipynb.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipynb.fs.full.CoefficientsCalcPlus import GetCoefficientsFlux
from ipynb.fs.full.CoefficientsCalcPlus import get_normalization
import sklearn.linear_model as linear_model
import scipy
import warnings; warnings.simplefilter('ignore')
import os
from matplotlib.pyplot import figure
import random as rnd
import matplotlib.ticker as ticker
#import cvxpy as cp
import uproot4 as uproot
import scipy.stats
import seaborn as sns
import sklearn
from matplotlib.colors import Normalize

In [ ]:
def get_normalization(coeffs):
    with uproot.open("/eos/home-a/amgruber/SWAN_projects/DUNEPRISM_XSec_SDClone_EditedVersion/Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians
    oa_flux = oa_flux.transpose()
    first_bin_flux = 8.9769702e-08
    norm = (oa_flux[0].sum()*0.01)/first_bin_flux
    oa_flux = oa_flux/norm
    print(oa_flux[0].sum()*0.01)
    oa_flux = oa_flux.transpose()
    lin_combo = np.matmul(oa_flux, coeffs)
    return lin_combo.sum()*0.01
def rebin(arr, factor):
    new_size = arr.size // factor
    remainder = arr.size % factor
    
    if remainder != 0:
        arr = arr[:-remainder]
        
    return np.sum(arr.reshape(-1, factor), axis=1)

def rebinned_bin_centers(nbins, xmin, xmax, factor):
    # Compute the bin widths for the original histogram
    bin_width = (xmax - xmin) / nbins
    
    # Compute the bin edges for the original histogram
    edges = np.linspace(xmin, xmax, nbins + 1)
    
    # Compute the new bin widths after rebinning
    new_bin_width = bin_width * factor
    
    # Compute the number of new bins
    new_nbins = int(nbins / factor)
    
    # Compute the bin edges for the rebinned histogram
    new_edges = np.linspace(xmin, xmax, new_nbins + 1)
    
    # Compute the bin centers for the rebinned histogram
    centers = (new_edges[1:] + new_edges[:-1]) / 2
    
    return centers

def shift(toy,energy):
    toy = np.roll(toy,energy,axis=-1)
    toy[..., -energy:] = 0
    return toy

#function definitions
def chi2sum(reco,folded,recoerror):
    res = ((np.array(reco)-np.array(folded))/np.array(recoerror))**2
    return res[res != np.inf].sum()

def c_norm(trueRidge):
    return np.sqrt((trueRidge**2).sum())

def GetGaussianRow(energy_bin_centers,target_loc,target_scale,rescaling = 1):
    return np.array([scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale) for x in energy_bin_centers])

def integrate(unfolded,rebin_factor):
    return np.array(unfolded).sum()*rebin_factor*0.001

def getAltSmearing(rebin=6):
    virtual_flux = [1.9544079707501092e-08, 2.6475454082874597e-07, 4.995168231442895e-07, 7.034347393767016e-07, 8.723231373511621e-07, 1.0327618514374411e-06, 1.2130213172365833e-06, 1.3077315488288394e-06, 1.3024199282868976e-06, 1.3462809582952103e-06, 1.4313642299314103e-06, 1.4837644027394322e-06, 1.6160466869142243e-06, 1.6737245427363842e-06, 1.8946772367527589e-06, 1.9502758352487968e-06, 2.1683976063375183e-06, 2.1943198860026654e-06, 2.3283211186830228e-06, 2.430133698358234e-06, 2.36477782098621e-06, 2.2833924585524217e-06, 2.149963043274324e-06, 2.066674127631667e-06, 1.7909791959821952e-06, 1.6528903858673855e-06, 1.4288297619954285e-06, 1.4210777138991235e-06, 1.3035570424204243e-06, 1.3557662254757407e-06, 1.3710682963246005e-06, 1.9724227599150446e-06, 3.0748126760349854e-06, 3.6724087671212137e-06, 4.000016762462219e-06, 3.7362812590423076e-06, 2.683198847262852e-06, 6.400757194577295e-07, -1.5678190799411306e-06, -3.737648983212363e-06, -5.547432738356778e-06, -6.504554905149321e-06, -6.874067095861655e-06, -6.138837873978747e-06, -5.770311546226911e-06, -4.541337934868358e-06, -2.4372590279078407e-06, -1.142031512637091e-06, 5.250302137158092e-07, 1.915014081952619e-06, 2.5812121596087064e-06, 2.703977313702166e-06, 2.860090881855267e-06, 2.581388204636515e-06, 2.6921971526365547e-06, 2.3799957005043744e-06, 2.6469198515050325e-06, 2.3023731554549112e-06, 3.4072710608037174e-06, 5.061830137498169e-06, 7.098571292813705e-06, 9.066042760552016e-06, 1.1903234130686772e-05, 1.570543134212948e-05, 1.9666611702192776e-05, 2.398749208572867e-05, 2.7524488735378383e-05, 3.2021263947853595e-05, 3.542623666168238e-05, 3.8640609517561825e-05, 4.1681748993381114e-05, 4.413669300451061e-05, 4.55045465564049e-05, 4.697560881005999e-05, 4.727112910015951e-05, 4.7027280765499577e-05, 4.567698959375994e-05, 4.405693694571463e-05, 4.200536659175848e-05, 3.9528605953484606e-05, 3.656488999248706e-05, 3.335168826101743e-05, 3.0438238527985208e-05, 2.7187974034614588e-05, 2.3925046231628316e-05, 2.0900337258855005e-05, 1.7509476877957283e-05, 1.5231490458486641e-05, 1.2444532707301437e-05, 9.924628514563359e-06, 8.066575902434062e-06, 6.069059869981723e-06, 4.570855862408862e-06, 2.8473723101564044e-06, 1.5686509405055974e-06, 7.398818137089891e-07, 2.0018173206142685e-07, -7.010315490789748e-07, -7.613331930764216e-07, -9.906631535177914e-07, -1.4815103587799028e-06, -1.587976550580772e-06, -1.7142305731565883e-06, -1.5293754543732406e-06, -1.6601715687737273e-06, -1.4486298953104618e-06, -1.2160208869805605e-06, -1.0859561186427698e-06, -1.3040140114510834e-06, -7.048759600200472e-07, -7.370812652538805e-07, -5.630087168838471e-07, -4.3085137047188567e-07, -2.919805239288291e-07, -1.6921173433924644e-07, 1.583152961053739e-07, -1.6928863117827033e-08, 1.691990888129371e-07, 4.2612193844126414e-07, 5.608309998881856e-07, 5.882071853357187e-07, 5.390868130402985e-07, 4.923335399547258e-07, 5.766731127433128e-07, 5.661115061625167e-07, 3.257512639066982e-07, 3.991434127611354e-07, 1.932844705322788e-07, 2.5726896092109794e-07, 1.6455035621201372e-07, 2.21416334058911e-08, 3.2429180772355793e-07, 3.078472626615721e-07, 1.2879700546589573e-07, 1.9023991384179473e-07, 1.872179227791859e-08, 1.7201299140488917e-07, 1.9064969840795584e-07, 1.3992022781137193e-07, -3.33100348332506e-08, -1.196098729424005e-07, 3.3615740681321444e-08, -3.668949241599281e-08, -8.152610735340473e-08, -4.96149558093258e-08, -9.149313463602981e-08, -1.6577625090072022e-09, 1.0943175791895585e-07, 6.250362555250303e-08, -4.389153678421679e-08, -6.494812188305432e-08, -1.7705927100864278e-07, 5.17549118471843e-08, 6.464289704840936e-08, -1.2602917803733108e-07, -4.994645028977069e-08, -3.48981011289187e-08, -2.4106705586706156e-07, -9.169861897830058e-08, -1.7829911284549898e-07, -8.892353373439835e-08, 3.939935500904217e-08, -9.841697648087526e-08, -1.7994105268519194e-07, -3.412534063952804e-07, -3.772139445921577e-07, -2.3778017280527098e-07, -2.919581263066587e-07, -3.769739865850758e-07, -6.706042860487262e-08, -2.4623787260212234e-07, -1.3211412136464202e-07, -3.7150697747220585e-07, -2.331652600732883e-07, -2.450948351347656e-07, -3.467696745187251e-07, -1.8456183640736922e-07, -1.749668243129279e-07, -1.7253399332124643e-07, -2.046652872957087e-07, -2.851483428565696e-07, -1.3614112135675682e-07, -1.0736722834411403e-07, -2.0300299313815918e-07, -1.911877547471585e-07, -1.557351354152551e-07, -6.385457377880248e-08, -1.7951928847825692e-07, -3.091613231220024e-07, -2.335391330278827e-07, -1.8511064706559747e-07, 5.1833515221051194e-08, -1.253121567541885e-07, -5.6694019126398155e-08, -1.5525826769516943e-07, -4.633480156912516e-09, 2.6011329037985096e-08, -1.7409087607528463e-07, 7.446987103891673e-08, 5.862064788866906e-08, 7.331811211712063e-08, 1.7151267009332474e-07, 1.5823485453763403e-07, -9.662392385785601e-08, -3.260761267668757e-08, 1.4528635106843745e-07, 1.391358647058318e-07, 1.392795462977539e-07, 2.763094686068247e-07, 1.8241264819781065e-07, 7.79860659821455e-08, 2.0602257371279083e-07, 1.5367967709223752e-07, 2.5023947132588553e-07, 1.0105846543369947e-07, 1.0867700066284062e-07, 2.196917092130851e-07, 1.547446343275892e-07, 1.437704176679805e-07, 2.1240058834394004e-07, 1.9511686609683506e-07, 2.530635709388644e-07, 2.0395770408109643e-07, 2.2559953893018906e-07, 1.889540557363441e-07, 1.4710588877213407e-07, 2.531709827588193e-07, 1.7371798586687818e-07, 8.47848004423449e-08, 1.5879315455908395e-07, 1.1627983308581763e-07, 2.5355795426992223e-09, 5.231980958149958e-08, 3.115287015764971e-08, 8.46302483368913e-08, 9.258689227679021e-08, -2.5443039979079568e-08, 2.723486626863733e-10, -6.398387141656718e-08, -2.229786240599873e-08, -1.1351433428240445e-07, -1.4217008419981186e-07, -9.79190381055561e-08, -7.560739473593266e-08, -7.57792346419855e-08, -1.2723123263073846e-08, -1.2852861518106352e-08, -1.2944694396250687e-07, -1.3781673061936732e-07, -1.2161024210622716e-08, -2.091267843740939e-07, -6.987826208061142e-08, -1.5281553984167436e-07, -1.4368721209698134e-07, -5.4680991323403114e-08, -2.022695413093557e-07, -1.969617689840062e-07, -2.395499699036155e-07, -1.28162813059118e-07, -1.6765662607327428e-07, -1.2937462531692373e-07, -1.3987163894490613e-07, -1.3460923795555446e-07, -4.493465427837363e-08, -1.759066216850725e-07, -5.209453399477118e-08, -1.5314513193810962e-07, -1.6052226885261853e-07, -7.552203910060381e-08, -9.180148897560504e-08, -1.5373085340033078e-07, -1.0466491101871176e-07, -1.1685124251722973e-07, -1.020824966655541e-07, -8.58256897665781e-08, -4.28698693138641e-08, -1.0328409060993384e-07, -1.1225197610476801e-07, -2.8890850628332537e-08, -1.3012155918029368e-07, -2.5247299466214068e-08, -9.249635158763944e-09, 2.683691242114861e-08, -5.3463007366126345e-08, 2.9584845184590532e-08, 4.042295986082495e-08, 1.3429728500127481e-08, 6.64387414877655e-08, 7.11883031047058e-08, 2.6550632849842396e-08, 7.066144498660552e-08, 3.9898639464878065e-08, 9.811946589046101e-08, 4.73363183008916e-08, 2.9782311185724585e-08, 1.7344221976087027e-08, 8.86281258620295e-08, 1.629496200558902e-07, 1.5609099636599233e-07, -4.7806569501388445e-09, 1.0225985832629676e-07, 8.754720174290366e-08, 1.1441401968912917e-07, 1.6361534611759235e-07, 2.0127918384570375e-07, 9.375843678410211e-08, 9.727583589720267e-08, 1.4632120324151797e-07, 1.800734475919106e-07, 5.9585348146751614e-08, -2.6076210012316172e-08, 1.4155947191757527e-07, 1.0450204801190688e-07, 1.5759233385567895e-07, 1.234637222364797e-07, 1.6162015666975512e-07, 1.773528893082807e-07, 3.304531188117047e-08, 6.053716925252364e-08, 7.812514218731014e-08, -1.723920597181587e-08, 1.127359991667336e-07, 9.802612268833905e-08, 6.802171623025582e-08, 7.008677248013925e-08, 8.669578668918477e-08, 2.9965549584688735e-08, 3.0149722150943746e-08, 7.89846440645447e-08, 1.1432292256494048e-07, 4.1930844451349496e-08, 3.40000582826206e-08, 6.618534450046538e-08, 7.310215144970023e-09, -2.7292826154378487e-10, 2.464718216525231e-09, -4.99784394992098e-08, -3.300363987290161e-08, 3.035368367973205e-08, 4.90552837909936e-08, -5.560072903194113e-08, -1.7736467902255096e-08, -8.309426113299321e-08, -4.027979749742655e-08, 4.3938041439437195e-08, -1.7745160269667367e-08, 2.994795345399951e-08, -8.810420814657891e-10, 3.20731594150189e-08, -6.543594885120125e-08, -1.3208380031888559e-07, -4.52372301903028e-08, -8.871071169926211e-09, 9.81030496934535e-09, -9.322433911518367e-08, -8.106401981065699e-09, -3.136842333786026e-08, -1.1635108424947325e-07, -9.124850247081637e-08, -9.330663382189973e-08, -3.9756146748145744e-08, -2.1438439210915506e-08, -6.70247958403775e-08, -8.280225921582046e-08, -3.6580433500525384e-08, -9.049132801837517e-08, 3.808762594977233e-08, -3.527342130219414e-08, 2.3016152210137672e-08, -2.5170448268942427e-08, -9.097313893242501e-09, -8.895609600143868e-08, -1.053199165138067e-08, 8.571773597029111e-09, -8.840416639746242e-09, 3.2512255220819364e-08, -4.033488463336833e-08, 1.5013480080089663e-08, -1.0580019041902985e-07, -9.99467281604833e-10, -5.954954256306013e-08, 1.0586219509595019e-08, 3.4363628117545174e-08, 9.82131103342557e-09, 5.402043092793124e-08, 4.6176341358426725e-08, 6.197302144484843e-08, 5.083840749871308e-08, 1.290959025841375e-07, 2.0573735422182742e-08, 3.539107132986933e-08, -7.06638161757471e-08, 5.771679402071777e-08, 4.053517392829467e-08, 7.082326569805016e-08, -3.2560276616539894e-08, 2.6929814586903627e-08, 4.727717521550324e-08, 5.31582264293114e-08, 1.0096852035329005e-07, 7.25076987445036e-08, 5.060590885281742e-08, 6.695703696870426e-08, 8.956826815527156e-09, 4.93554061533321e-08, 5.778929766097648e-08, 2.9661903802150402e-08, 3.872898792477724e-08, 6.13017372640752e-08, 7.532191402882403e-09, 4.047719211395082e-08, 4.001256338707339e-10, -5.737478492214294e-09, -1.9816630428450102e-09, -2.426367004836114e-09, 2.782005414627754e-08, 1.4466729913156034e-08, -3.320269993145339e-09, 6.148586833355686e-10, 9.017650209865869e-09, -1.5781753497278517e-08, -7.976767378175333e-08, -7.382041661577098e-08, -3.4533158822485614e-08, -7.228762964096421e-08, -1.1011118027938165e-07, -6.880974672463053e-08, -9.244967005530784e-08, -6.267926351121554e-08, -3.8869122383601086e-08, -1.3432765512404446e-07, -1.2643071356231653e-07, -1.356085455756915e-07, -1.3536019022907957e-07, -1.0595489415308299e-07, -1.6191390264687707e-07, -1.6441013112310997e-07, -1.3978512550626832e-07, -2.4012314611764254e-07, -2.3699921395937074e-07, -2.0949258760253627e-07, -2.108314790283951e-07, -2.145832053292722e-07, -1.906359969931756e-07, -2.5668973636919884e-07, -2.600561631513878e-07, -1.8164789975077183e-07, -2.1327152655744062e-07, -2.57907101793626e-07, -2.6506566865566495e-07, -2.521925449979643e-07, -2.828647604937902e-07, -2.426343618571237e-07, -2.2362304940470716e-07, -2.989789086896128e-07, -3.0109197246159914e-07, -2.2183239639120585e-07, -2.3861747833862843e-07, -2.642847221727079e-07, -3.0807089973724986e-07, -2.840335165412386e-07, -2.715809303316445e-07, -2.709605376955467e-07, -2.2680800614121508e-07, -2.2335273786716435e-07, -2.8379883469115267e-07, -2.1406694145552387e-07, -2.6547164649919765e-07, -2.4862122379889924e-07, -2.2010531084523947e-07, -2.492609947550892e-07, -2.1249402943797244e-07, -2.0353284306511572e-07, -2.2872394858268463e-07, -2.1524857183947562e-07, -2.3959672676984075e-07, -1.5013312652124056e-07, -2.1233210820226392e-07, -1.8937813822655293e-07, -1.4987144010979504e-07, -9.089000780351273e-08, -2.0831711177126075e-07, -1.4549236703709716e-07, -1.0853284218201223e-07, -7.92308701089751e-08, -1.5362094607231444e-07, -1.0484819790456546e-07, -4.591951580139733e-08, -4.480631449574848e-08, -5.096497772955719e-08, -4.465991615866574e-08, -3.3786824179908514e-08, 5.55326571813132e-09, -1.5402279758805646e-08, 5.772234023639118e-08, 2.2396637489979832e-10, 5.151332782292962e-08, 4.333133502683795e-08, 5.988072961720329e-08, 8.575300639389256e-08, 6.704302784494715e-08, 1.1532555026669435e-07, 1.0859971704918264e-07, 1.6758727937418788e-07, 1.6679416968233936e-07, 1.9088424173875684e-07, 2.2505311856025573e-07, 2.0581915687166248e-07, 1.7111411511398223e-07, 2.0210393232633052e-07, 2.196861902635954e-07, 2.1334143812767434e-07, 2.385474270883562e-07, 2.969172987775183e-07, 2.635732693693258e-07, 2.902309141765205e-07, 2.9439594523747595e-07, 2.8803084195106686e-07, 2.926340237575644e-07, 3.370010491698146e-07, 3.530599261634766e-07, 3.230348550656637e-07, 3.6036249663200583e-07, 4.0363616716743597e-07, 3.959783079413646e-07, 3.608221955500643e-07, 3.703044651882817e-07, 3.7492344203911674e-07, 3.669808637789546e-07, 4.1181449450194085e-07, 4.264665158975321e-07, 4.290350144391984e-07, 4.378054825656196e-07, 3.948033601361643e-07, 4.6124623795198286e-07, 4.425121031241174e-07, 4.4518457960982514e-07, 4.811667731922438e-07, 4.560150941941316e-07, 4.906444189740838e-07, 4.524778329862562e-07, 4.606497353497653e-07, 4.714308804642767e-07, 4.823954586034293e-07, 5.014715336448452e-07, 4.884725628445244e-07, 4.909255231668533e-07, 4.6734288000639967e-07, 4.2745309988286355e-07, 4.768701197856387e-07, 4.886682563103507e-07, 4.5466159840720214e-07, 4.64556972311244e-07, 4.5430673952421813e-07, 4.4290985955943806e-07, 4.7679197427267275e-07, 4.344035784917916e-07, 4.647027439675095e-07, 4.5428665505358565e-07, 4.825635889166459e-07, 4.624059664197144e-07, 4.888492770686605e-07, 4.5932131485934427e-07, 4.1771033177751477e-07, 4.286138547137977e-07, 4.1573628538712165e-07, 4.0247279342443555e-07, 4.139151771734614e-07, 4.1741729244043195e-07, 3.8120028496929746e-07, 4.1028919543853913e-07, 3.9858556043241494e-07, 4.166829889855156e-07, 3.552121040510905e-07, 4.227448734159322e-07, 3.988646210439181e-07, 3.7320980374358377e-07, 3.673420536165784e-07, 3.6166717823749773e-07, 3.234210385203521e-07, 3.799455792297658e-07, 3.5404326289176257e-07, 3.3207153642660237e-07, 2.8158368136310955e-07, 3.2889019676127314e-07, 2.985262562352499e-07, 3.0358640271041215e-07, 2.9270780326473327e-07, 2.4707781383996167e-07, 3.0318816992060033e-07, 2.762155749239446e-07, 2.6988828518455727e-07, 2.8340386309240324e-07, 2.783704373327563e-07, 2.4918983474416255e-07, 2.844044154229286e-07, 2.531492165931901e-07, 2.802533087927208e-07, 2.393712848496859e-07, 2.4991333307910606e-07, 2.377465010968621e-07, 2.0851572537909825e-07, 2.193432736664815e-07, 2.1461985347265648e-07, 1.7892258860387095e-07, 1.9350690774769848e-07, 1.9951824921811393e-07, 1.847426663389758e-07, 1.8577464348527715e-07, 1.830498547799969e-07, 1.9855170545383174e-07, 1.8079174516637217e-07, 1.7628115914608386e-07, 2.068276718051054e-07, 1.9275833783313538e-07, 1.5076063281751946e-07, 1.508668268902724e-07, 1.5211277368457883e-07, 1.557769474595891e-07, 1.4324779061554426e-07, 1.2394190313135573e-07, 1.3123485774155708e-07, 1.3404079977206636e-07, 1.3165669400273668e-07, 1.0226516753029672e-07, 1.4324431546957356e-07, 1.0489908158414491e-07, 1.4802287936130708e-07, 1.185531765589464e-07, 1.3997394201223707e-07, 1.1888761555757006e-07, 1.1055073141717556e-07, 1.099761143060462e-07, 1.2131724565979403e-07, 1.1044620355865387e-07, 1.0485434142359052e-07, 1.0613752130862281e-07, 4.283993144697393e-08, 1.0755005578448653e-07, 9.387712244238844e-08, 1.1345961784665479e-07, 6.796686457908833e-08, 1.0836123026164256e-07, 1.023858996789179e-07, 1.1457049310805159e-07, 9.739776641766409e-08, 8.883105712273107e-08, 1.00520928211047e-07, 1.2528365667107246e-07, 1.000923058090771e-07, 1.1901730913688088e-07, 1.1818537120118581e-07, 9.743528355928076e-08, 1.3134389171828861e-07, 9.382213980063038e-08, 1.147592472531976e-07, 7.682010425952981e-08, 9.172963647134887e-08, 8.222342722511688e-08, 7.83125535015067e-08, 8.079814899420654e-08, 9.257867648219796e-08, 8.523169972628612e-08, 1.2205091568967878e-07, 9.318630386012542e-08, 9.9388148700089e-08, 1.2872732706078336e-07, 1.012413079094425e-07, 6.977123600105178e-08, 1.0372259672129317e-07, 9.175749761243992e-08, 1.0759363618775414e-07, 9.532043731656528e-08, 9.387860798245794e-08, 7.634307472186546e-08, 1.3366654913329597e-07, 1.1216402474110656e-07, 8.655028644755713e-08, 9.479921121209881e-08, 9.00444110460896e-08, 8.1701758054753e-08, 1.5736882249533005e-07, 8.490500532176676e-08, 1.5340671641074583e-07, 1.153449495708721e-07, 1.0406503006487235e-07, 1.2184721802073472e-07, 1.1892198020187171e-07, 7.715656238024353e-08, 1.0115706351405115e-07, 1.1903927742788805e-07, 1.0440957034539646e-07, 7.371094787149768e-08, 1.1405719248174044e-07, 1.1781398254065359e-07, 1.3623468035064888e-07, 8.932497946789629e-08, 1.174528366477442e-07, 1.3215107675242865e-07, 9.490902137713101e-08, 1.033935138541733e-07, 1.0853338324723975e-07, 1.333336886048846e-07, 1.3820646595090328e-07, 1.2717417174617928e-07, 9.680324712186004e-08, 1.190387356457267e-07, 1.0656217120444046e-07, 1.2856926114157773e-07, 1.2899003677556326e-07, 1.3136059552924267e-07, 9.518787690363736e-08, 1.1586721966886445e-07, 1.1769150334472414e-07, 1.2149194051709054e-07, 1.327435351067283e-07, 1.1571265103969419e-07, 1.223523540319312e-07, 1.2817592291574982e-07, 1.0362867514452969e-07, 9.343025154995784e-08, 1.3130320264191258e-07, 1.529259528919034e-07, 1.3575825822407423e-07, 1.1803040812523376e-07, 1.301385524423545e-07, 1.5485507477351353e-07, 1.1742572017265192e-07, 1.293475953288645e-07, 1.1354358771706547e-07, 1.1425783150846433e-07, 1.3633383750660223e-07, 1.0366057062602598e-07, 1.3038964103043732e-07, 1.2050225379566283e-07, 8.667672212473778e-08, 1.324527251772468e-07, 1.3255690231986325e-07, 8.727956089266263e-08, 1.2908532963349342e-07, 1.3259068502283036e-07, 1.411600079599528e-07, 1.3551145878014005e-07, 1.325125750533111e-07, 1.3000098371194474e-07, 1.2412737385938373e-07, 9.328939450315852e-08, 1.2365461619568345e-07, 1.2955891516012336e-07, 1.1676823308546029e-07, 1.131646817296374e-07, 1.2111844862726863e-07, 1.1065367539561688e-07, 1.3220071056698995e-07, 9.702077525191118e-08, 1.206907125699916e-07, 1.201038328029265e-07, 1.0635498956049528e-07, 1.4731295197148215e-07, 1.2074588468224472e-07, 1.252260812202951e-07, 1.151130152316468e-07, 1.1724701743827123e-07, 1.0849730164213482e-07, 9.959620247604396e-08, 8.952353365201146e-08, 1.319616394707839e-07, 1.1446155988296472e-07, 1.182164750167469e-07, 1.1897849757671372e-07, 1.168911181213612e-07, 1.147442505966229e-07, 1.241447805879349e-07, 1.1162946488732498e-07, 1.0277029970040184e-07, 1.0645661757356271e-07, 1.037037660096789e-07, 1.1864689491942077e-07, 1.0372249841048991e-07, 1.2385146235095566e-07, 5.350134046542605e-08, 1.1568045399574903e-07, 1.0180306037394498e-07, 1.0471896183780163e-07, 9.588500041204355e-08, 8.023209964815496e-08, 8.584059477764237e-08, 1.0029873043320375e-07, 7.793056181088224e-08, 1.0707105914511864e-07, 9.778069830884617e-08, 1.2120167061806175e-07, 1.23317260288583e-07, 1.2606096797954097e-07, 8.523070854064114e-08, 1.0459737719879438e-07, 9.002764950270196e-08, 1.046893704398055e-07, 9.40606940479623e-08, 1.1739204507179368e-07]
    virtual_flux = [0 for i in range(100)]+virtual_flux
    virtual_flux = [np.array(virtual_flux[rebin*i:rebin*i+rebin]).sum() for i in range(int(len(virtual_flux)/rebin)-1)]
    plt.plot(virtual_flux)

    energies = [-1+(rebin*0.01)/2+rebin*0.01*i for i in range(int(len(virtual_flux)))]
    max_index = virtual_flux.index(max(virtual_flux))
    virtual_flux = np.array(virtual_flux)
    virtual_flux = virtual_flux/virtual_flux.sum()
    virtual_flux = list(virtual_flux)
    matrix = []
    for j in range(149):
        row = []
        if j < max_index:
            row = virtual_flux[max_index-j:len(energies)] + [0 for i in range(max_index-j)]
        else:
            row = [0 for i in range(j-max_index)] + virtual_flux[0:len(energies)-j+max_index]
        matrix.append(row)
    return matrix

def curvature(x_ridge,y_ridge,alphas):
    fac = 1.

    res = x_ridge
    sol = y_ridge
    dl = np.diff(alphas)
    xi = np.log(sol)
    rho = np.log(res)
    xi_prime = (1/fac)*np.diff(xi)/dl
    rho_prime = np.diff(rho)/dl
    xi_prime_prime = fac*np.diff(xi_prime)/dl[:-1]
    rho_prime_prime = np.diff(rho_prime)/dl[:-1]

    curv = 2*(rho_prime[:-1]*xi_prime_prime - rho_prime_prime*xi_prime[:-1])/np.power(np.power(rho_prime[:-1], 2) + np.power(xi_prime[:-1], 2), 3./2)

    curvature = plt.plot(alphas[1:-1], curv, "r", label = "Curvature")
    plt.xlabel("Regularisation strength")
    plt.ylabel("L-curve Curvature")
    plt.xscale("log")
    plt.legend();
    plt.show(curvature)
    
    maxCurv = np.max(curv[~np.isnan(curv)])
    num = alphas[1:-1][curv == maxCurv][0]
    print(f'Optimal regularisation strength = {num}')
    return num

def M_shape(M,N):
    first = (N/N.sum())*M.sum(axis=1, keepdims=True)
    second = (N[:, np.newaxis]/N.sum())*M.sum(axis=0, keepdims=True)
    third = (N*N[:, np.newaxis]/N.sum()**2) * M.sum()
    return M - first - second + third

def M_mixed(M,N):
    first = (N/N.sum())*M.sum(axis=1, keepdims=True)
    second = (N[:, np.newaxis]/N.sum())*M.sum(axis=0, keepdims=True)
    third = (N*N[:, np.newaxis]/N.sum()**2) * M.sum() 
    return first + second - 2*third

def M_norm(M,N):
    third = (N*N[:, np.newaxis]/N.sum()**2) * M.sum()
    return third

def get_gaussian_std(prediction,energy_bin_centers,oa_events):
    gfit = optimize.curve_fit(gaussian,energy_bin_centers,prediction)[0]
    loc,scale = gfit[0],gfit[1]
    return [scale,loc]

def gaussian(x,loc,scale):
    return (1/(scale*((2*np.pi)**0.5)))*np.exp((-((x-loc)/scale)**2)/2)*rescaling

def get_virtual_flux(energy,width,plot=True):
    # Get DUNE near detector muon neutrino flux as a function of off-axis angle
    with uproot.open("/eos/home-a/amgruber/SWAN_projects/DUNEPRISM_XSec_SDClone_EditedVersion/Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians

    # From oa_flux in [neutrinos cm^-2 per POT per year] to oa_events in [neutrinos per GeV per Year]
    #cross_section = 10e-38*np.array([1.7564017,1.7524341,1.7496688,1.7444551,1.7370047,1.7231702,1.7069778,1.6844808,1.6565275,1.6218245,1.5793541,1.5299167,1.4797076,1.4227808,1.3687412,1.3173940,1.2662984,1.2205544,1.1756606,1.1365877,1.1000308,1.0661938,1.0359266,1.0081116,0.98149143,0.95945773,0.93715417,0.91382316,0.89369118,0.87390051,0.85512638,0.83607184,0.81882318,0.80275432,0.78789418,0.76993262,0.75490589,0.74114020,0.72626038,0.71124357,0.69732170,0.68273972,0.66883140,0.65590717,0.64289072,0.62991282,0.61805200,0.60649662,0.59396039,0.58324116,0.57177118,0.56149453,0.55099552,0.54106390,0.53076420,0.52044680,0.51228947]) #[cm^2 nucleon^-1]
    xsecs_string = open("xsecs.txt", "r").read().split("\n")
    cross_section = 1e-38*np.array([float(x) for x in xsecs_string])
    LFGXsec = [1.7949623100383943e-38, 1.7937963091278917e-38, 1.7911772525412136e-38, 1.7861854895796225e-38, 1.7809750967678895e-38, 1.7701494830989152e-38, 1.7545813390805861e-38, 1.737516314683793e-38, 1.7115823937822531e-38, 1.6803055726248326e-38, 1.6417743167096645e-38, 1.593913213540418e-38, 1.5450129113507074e-38, 1.4886002213287708e-38, 1.4345628311913895e-38, 1.3812337472883923e-38, 1.331042079303393e-38, 1.2815197183341513e-38, 1.2363422627087503e-38, 1.1935201632601205e-38, 1.1549648479647322e-38, 1.1203762360503803e-38, 1.0889404829943938e-38, 1.0592474272370098e-38, 1.0315306729032208e-38, 1.0062724029127059e-38, 9.820934128895517e-39, 9.589284572449444e-39, 9.377202159721729e-39, 9.17570574901543e-39, 8.965063741216594e-39, 8.762369716805595e-39, 8.58922826685481e-39, 8.41909721459255e-39, 8.248714544179755e-39, 8.068406642454413e-39, 7.906157098609457e-39, 7.751083746194647e-39, 7.597402661066086e-39, 7.447370602905079e-39, 7.289566752473818e-39, 7.147988243981008e-39, 6.994996911117083e-39, 6.858379706916821e-39, 6.715880715605158e-39, 6.573442019863122e-39, 6.447041269811119e-39, 6.316099814217903e-39, 6.197192628264873e-39, 6.072034334669806e-39, 5.961443680046778e-39, 5.8364518213083814e-39, 5.727036266656917e-39, 5.627020176292163e-39, 5.510771609161703e-39, 5.419195161107539e-39, 5.3170150477088846e-39, 5.223406136588331e-39]
    #SFXsec = [1.754550404489065e-44, 1.7519613275809567e-44, 1.750979572129333e-44, 1.7473270318264672e-44, 1.7405225150746676e-44, 1.7319576073355019e-44, 1.71692135094249e-44, 1.6968159242111131e-44, 1.6728160715962411e-44, 1.64139412647603e-44, 1.6006334393659204e-44, 1.558043745204266e-44, 1.5020221621743717e-44, 1.4474714840058103e-44, 1.393084987535154e-44, 1.3415745131766955e-44, 1.2891072990176513e-44, 1.2401131409657053e-44, 1.1957213224761003e-44, 1.1555398112868075e-44, 1.11735950056154e-44, 1.0837463293428634e-44, 1.0508735469225472e-44, 1.0210651058040137e-44, 9.948296423377384e-45, 9.707225864225561e-45, 9.464105938772724e-45, 9.239051192689146e-45, 9.022692739181657e-45, 8.833035380131772e-45, 8.640216984303538e-45, 8.448725011633172e-45, 8.281463202564871e-45, 8.10007897291191e-45, 7.941373143181159e-45, 7.795048546210453e-45, 7.630944681755204e-45, 7.471497099332307e-45, 7.327591393984823e-45, 7.183129575522519e-45, 7.04901578190321e-45, 6.894217398446113e-45, 6.768880999253077e-45, 6.626102699712311e-45, 6.499500747453237e-45, 6.359158836942974e-45, 6.239597461445368e-45, 6.116790965334497e-45, 5.999856815597659e-45, 5.876192571794982e-45, 5.76950347360696e-45, 5.667004567377422e-45, 5.556517121700038e-45, 5.459224582089677e-45, 5.3609214282572393e-45, 5.25696243349593e-45, 5.1666536593894114e-45, 5.0794279523476983e-45]
    N_target = 1.435e30 #[nucleon]
    E = 3.62e19 #[POT per year]
    epsilon = 1
    cross_section = np.tile(cross_section, (oa_flux.shape[0],1))
    LFGXsec = np.tile(LFGXsec, (oa_flux.shape[0],1))
    oa_events = oa_flux * cross_section * epsilon * E * N_target * 1e-2
    LFG_oa_events = oa_flux * LFGXsec * epsilon * E * N_target * 1e-2
    events_vector = np.array([oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])
    LFG_events_vector = np.array([LFG_oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])

    target_loc = 1e-3*energy
    target_scale = 1e-3*width #change this back to 0.1
    rescaling = oa_flux.max()
    print(rescaling)
    #rescaling = 1
    energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:])/2.
    target_flux = [scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale)*rescaling for x in energy_bin_centers] #here we should scale the gaussian

    clf = linear_model.Ridge(alpha=1e-12,fit_intercept=False) #when fit_intercept=False OA(dot)x = predict(x)
    clf.fit(oa_flux,target_flux)
    x_ridge = clf.coef_

    if plot==True:
        #figure(figsize=(8, 6))
        plt.plot(energy_bin_centers, target_flux, label = "Target Flux")
        plt.plot(energy_bin_centers, clf.predict(oa_flux), label = "Virtual Flux")
        plt.xlabel(r"E$_{\nu}$ [GeV]")
        #plt.xlim([0, 1.5])

        plt.legend();
        plt.grid(True, color = "grey", linewidth = "1", linestyle = "-")
        plt.show()

def get_gaussian_std(prediction,energy_bin_centers,oa_events):
    gfit = optimize.curve_fit(gaussian,energy_bin_centers,prediction)[0]
    loc,scale = gfit[0],gfit[1]
    return [scale,loc]

def gaussian(x,loc,scale):
    return (1/(scale*((2*np.pi)**0.5)))*np.exp((-((x-loc)/scale)**2)/2)*rescaling

def get_virtual_flux(energy,width,plot=True):
    # Get DUNE near detector muon neutrino flux as a function of off-axis angle
    with uproot.open("/eos/home-a/amgruber/SWAN_projects/DUNEPRISM_XSec_SDClone_EditedVersion/Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians

    # From oa_flux in [neutrinos cm^-2 per POT per year] to oa_events in [neutrinos per GeV per Year]
    #cross_section = 10e-38*np.array([1.7564017,1.7524341,1.7496688,1.7444551,1.7370047,1.7231702,1.7069778,1.6844808,1.6565275,1.6218245,1.5793541,1.5299167,1.4797076,1.4227808,1.3687412,1.3173940,1.2662984,1.2205544,1.1756606,1.1365877,1.1000308,1.0661938,1.0359266,1.0081116,0.98149143,0.95945773,0.93715417,0.91382316,0.89369118,0.87390051,0.85512638,0.83607184,0.81882318,0.80275432,0.78789418,0.76993262,0.75490589,0.74114020,0.72626038,0.71124357,0.69732170,0.68273972,0.66883140,0.65590717,0.64289072,0.62991282,0.61805200,0.60649662,0.59396039,0.58324116,0.57177118,0.56149453,0.55099552,0.54106390,0.53076420,0.52044680,0.51228947]) #[cm^2 nucleon^-1]
    xsecs_string = open("xsecs.txt", "r").read().split("\n")
    cross_section = 1e-38*np.array([float(x) for x in xsecs_string])
    LFGXsec = [1.7949623100383943e-38, 1.7937963091278917e-38, 1.7911772525412136e-38, 1.7861854895796225e-38, 1.7809750967678895e-38, 1.7701494830989152e-38, 1.7545813390805861e-38, 1.737516314683793e-38, 1.7115823937822531e-38, 1.6803055726248326e-38, 1.6417743167096645e-38, 1.593913213540418e-38, 1.5450129113507074e-38, 1.4886002213287708e-38, 1.4345628311913895e-38, 1.3812337472883923e-38, 1.331042079303393e-38, 1.2815197183341513e-38, 1.2363422627087503e-38, 1.1935201632601205e-38, 1.1549648479647322e-38, 1.1203762360503803e-38, 1.0889404829943938e-38, 1.0592474272370098e-38, 1.0315306729032208e-38, 1.0062724029127059e-38, 9.820934128895517e-39, 9.589284572449444e-39, 9.377202159721729e-39, 9.17570574901543e-39, 8.965063741216594e-39, 8.762369716805595e-39, 8.58922826685481e-39, 8.41909721459255e-39, 8.248714544179755e-39, 8.068406642454413e-39, 7.906157098609457e-39, 7.751083746194647e-39, 7.597402661066086e-39, 7.447370602905079e-39, 7.289566752473818e-39, 7.147988243981008e-39, 6.994996911117083e-39, 6.858379706916821e-39, 6.715880715605158e-39, 6.573442019863122e-39, 6.447041269811119e-39, 6.316099814217903e-39, 6.197192628264873e-39, 6.072034334669806e-39, 5.961443680046778e-39, 5.8364518213083814e-39, 5.727036266656917e-39, 5.627020176292163e-39, 5.510771609161703e-39, 5.419195161107539e-39, 5.3170150477088846e-39, 5.223406136588331e-39]
    #SFXsec = [1.754550404489065e-44, 1.7519613275809567e-44, 1.750979572129333e-44, 1.7473270318264672e-44, 1.7405225150746676e-44, 1.7319576073355019e-44, 1.71692135094249e-44, 1.6968159242111131e-44, 1.6728160715962411e-44, 1.64139412647603e-44, 1.6006334393659204e-44, 1.558043745204266e-44, 1.5020221621743717e-44, 1.4474714840058103e-44, 1.393084987535154e-44, 1.3415745131766955e-44, 1.2891072990176513e-44, 1.2401131409657053e-44, 1.1957213224761003e-44, 1.1555398112868075e-44, 1.11735950056154e-44, 1.0837463293428634e-44, 1.0508735469225472e-44, 1.0210651058040137e-44, 9.948296423377384e-45, 9.707225864225561e-45, 9.464105938772724e-45, 9.239051192689146e-45, 9.022692739181657e-45, 8.833035380131772e-45, 8.640216984303538e-45, 8.448725011633172e-45, 8.281463202564871e-45, 8.10007897291191e-45, 7.941373143181159e-45, 7.795048546210453e-45, 7.630944681755204e-45, 7.471497099332307e-45, 7.327591393984823e-45, 7.183129575522519e-45, 7.04901578190321e-45, 6.894217398446113e-45, 6.768880999253077e-45, 6.626102699712311e-45, 6.499500747453237e-45, 6.359158836942974e-45, 6.239597461445368e-45, 6.116790965334497e-45, 5.999856815597659e-45, 5.876192571794982e-45, 5.76950347360696e-45, 5.667004567377422e-45, 5.556517121700038e-45, 5.459224582089677e-45, 5.3609214282572393e-45, 5.25696243349593e-45, 5.1666536593894114e-45, 5.0794279523476983e-45]
    N_target = 1.435e30 #[nucleon]
    E = 3.62e19 #[POT per year]
    epsilon = 1
    cross_section = np.tile(cross_section, (oa_flux.shape[0],1))
    LFGXsec = np.tile(LFGXsec, (oa_flux.shape[0],1))
    oa_events = oa_flux * cross_section * epsilon * E * N_target * 1e-2
    LFG_oa_events = oa_flux * LFGXsec * epsilon * E * N_target * 1e-2
    events_vector = np.array([oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])
    LFG_events_vector = np.array([LFG_oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])

    target_loc = 1e-3*energy
    target_scale = 1e-3*width #change this back to 0.1
    rescaling = oa_flux.max()
    print(rescaling)
    #rescaling = 1
    energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:])/2.
    target_flux = [scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale)*rescaling for x in energy_bin_centers] #here we should scale the gaussian

    clf = linear_model.Ridge(alpha=1e-12,fit_intercept=False) #when fit_intercept=False OA(dot)x = predict(x)
    clf.fit(oa_flux,target_flux)
    x_ridge = clf.coef_

    if plot==True:
        #figure(figsize=(8, 6))
        plt.plot(energy_bin_centers, target_flux, label = "Target Flux")
        plt.plot(energy_bin_centers, clf.predict(oa_flux), label = "Virtual Flux")
        plt.xlabel(r"E$_{\nu}$ [GeV]")
        #plt.xlim([0, 1.5])

        plt.legend();
        plt.grid(True, color = "grey", linewidth = "1", linestyle = "-")
        plt.show()

def get_virtual_flux(energy,width,alpha_val=1e-12,plot=True):
    # Get DUNE near detector muon neutrino flux as a function of off-axis angle
    with uproot.open("/eos/home-a/amgruber/SWAN_projects/DUNEPRISM_XSec_SDClone_EditedVersion/Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians

    # From oa_flux in [neutrinos cm^-2 per POT per year] to oa_events in [neutrinos per GeV per Year]
    #cross_section = 10e-38*np.array([1.7564017,1.7524341,1.7496688,1.7444551,1.7370047,1.7231702,1.7069778,1.6844808,1.6565275,1.6218245,1.5793541,1.5299167,1.4797076,1.4227808,1.3687412,1.3173940,1.2662984,1.2205544,1.1756606,1.1365877,1.1000308,1.0661938,1.0359266,1.0081116,0.98149143,0.95945773,0.93715417,0.91382316,0.89369118,0.87390051,0.85512638,0.83607184,0.81882318,0.80275432,0.78789418,0.76993262,0.75490589,0.74114020,0.72626038,0.71124357,0.69732170,0.68273972,0.66883140,0.65590717,0.64289072,0.62991282,0.61805200,0.60649662,0.59396039,0.58324116,0.57177118,0.56149453,0.55099552,0.54106390,0.53076420,0.52044680,0.51228947]) #[cm^2 nucleon^-1]
    xsecs_string = open("xsecs.txt", "r").read().split("\n")
    cross_section = 1e-38*np.array([float(x) for x in xsecs_string])
    LFGXsec = [1.7949623100383943e-38, 1.7937963091278917e-38, 1.7911772525412136e-38, 1.7861854895796225e-38, 1.7809750967678895e-38, 1.7701494830989152e-38, 1.7545813390805861e-38, 1.737516314683793e-38, 1.7115823937822531e-38, 1.6803055726248326e-38, 1.6417743167096645e-38, 1.593913213540418e-38, 1.5450129113507074e-38, 1.4886002213287708e-38, 1.4345628311913895e-38, 1.3812337472883923e-38, 1.331042079303393e-38, 1.2815197183341513e-38, 1.2363422627087503e-38, 1.1935201632601205e-38, 1.1549648479647322e-38, 1.1203762360503803e-38, 1.0889404829943938e-38, 1.0592474272370098e-38, 1.0315306729032208e-38, 1.0062724029127059e-38, 9.820934128895517e-39, 9.589284572449444e-39, 9.377202159721729e-39, 9.17570574901543e-39, 8.965063741216594e-39, 8.762369716805595e-39, 8.58922826685481e-39, 8.41909721459255e-39, 8.248714544179755e-39, 8.068406642454413e-39, 7.906157098609457e-39, 7.751083746194647e-39, 7.597402661066086e-39, 7.447370602905079e-39, 7.289566752473818e-39, 7.147988243981008e-39, 6.994996911117083e-39, 6.858379706916821e-39, 6.715880715605158e-39, 6.573442019863122e-39, 6.447041269811119e-39, 6.316099814217903e-39, 6.197192628264873e-39, 6.072034334669806e-39, 5.961443680046778e-39, 5.8364518213083814e-39, 5.727036266656917e-39, 5.627020176292163e-39, 5.510771609161703e-39, 5.419195161107539e-39, 5.3170150477088846e-39, 5.223406136588331e-39]
    #SFXsec = [1.754550404489065e-44, 1.7519613275809567e-44, 1.750979572129333e-44, 1.7473270318264672e-44, 1.7405225150746676e-44, 1.7319576073355019e-44, 1.71692135094249e-44, 1.6968159242111131e-44, 1.6728160715962411e-44, 1.64139412647603e-44, 1.6006334393659204e-44, 1.558043745204266e-44, 1.5020221621743717e-44, 1.4474714840058103e-44, 1.393084987535154e-44, 1.3415745131766955e-44, 1.2891072990176513e-44, 1.2401131409657053e-44, 1.1957213224761003e-44, 1.1555398112868075e-44, 1.11735950056154e-44, 1.0837463293428634e-44, 1.0508735469225472e-44, 1.0210651058040137e-44, 9.948296423377384e-45, 9.707225864225561e-45, 9.464105938772724e-45, 9.239051192689146e-45, 9.022692739181657e-45, 8.833035380131772e-45, 8.640216984303538e-45, 8.448725011633172e-45, 8.281463202564871e-45, 8.10007897291191e-45, 7.941373143181159e-45, 7.795048546210453e-45, 7.630944681755204e-45, 7.471497099332307e-45, 7.327591393984823e-45, 7.183129575522519e-45, 7.04901578190321e-45, 6.894217398446113e-45, 6.768880999253077e-45, 6.626102699712311e-45, 6.499500747453237e-45, 6.359158836942974e-45, 6.239597461445368e-45, 6.116790965334497e-45, 5.999856815597659e-45, 5.876192571794982e-45, 5.76950347360696e-45, 5.667004567377422e-45, 5.556517121700038e-45, 5.459224582089677e-45, 5.3609214282572393e-45, 5.25696243349593e-45, 5.1666536593894114e-45, 5.0794279523476983e-45]
    N_target = 1.435e30 #[nucleon]
    E = 3.62e19 #[POT per year]
    epsilon = 1
    cross_section = np.tile(cross_section, (oa_flux.shape[0],1))
    LFGXsec = np.tile(LFGXsec, (oa_flux.shape[0],1))
    oa_events = oa_flux * cross_section * epsilon * E * N_target * 1e-2
    LFG_oa_events = oa_flux * LFGXsec * epsilon * E * N_target * 1e-2
    events_vector = np.array([oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])
    LFG_events_vector = np.array([LFG_oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])

    target_loc = 1e-3*energy
    target_scale = 1e-3*width #change this back to 0.1
    rescaling = oa_flux.max()
    print(rescaling)
    #rescaling = 1
    energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:])/2.
    target_flux = [scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale)*rescaling for x in energy_bin_centers] #here we should scale the gaussian

    clf = linear_model.Ridge(alpha=alpha_val,fit_intercept=False) #when fit_intercept=False OA(dot)x = predict(x)
    clf.fit(oa_flux,target_flux)
    x_ridge = clf.coef_

    if plot==True:
        #figure(figsize=(8, 6))
        plt.plot(energy_bin_centers, target_flux, label = "Target Flux")
        plt.plot(energy_bin_centers, clf.predict(oa_flux), label = "Virtual Flux")
        plt.xlabel(r"E$_{\nu}$ [GeV]")
        #plt.xlim([0, 1.5])

        plt.legend();
        plt.grid(True, color = "grey", linewidth = "1", linestyle = "-")
        plt.show()
    return clf.predict(oa_flux)

def virtual_smearing(energy,width,alpha,rebin_factor):
    virtual_flux = get_virtual_flux(energy,width,alpha,plot=False)
    virtual_flux = rebin(virtual_flux,rebin_factor//10)
    padding = [0 for i in range(len(virtual_flux))]
    virtual_flux = np.concatenate((padding,virtual_flux,padding,padding))
    print(virtual_flux.shape)
    virtual_flux = np.array(virtual_flux)
    flux_max = np.argmax(virtual_flux)
    
    total_bins = 16000//rebin_factor
    normalization = virtual_flux.sum()
    smearing = np.array([virtual_flux[flux_max+total_bins//2-i:flux_max+total_bins//2-i+total_bins//2] for i in range(total_bins)])/normalization
    return smearing
    

def extract_blocks(matrix, block_indices):
    """
    Extracts and combines specified blocks from a matrix into a new matrix.

    Args:
        matrix (numpy.ndarray): The input matrix.
        block_indices (list of tuples): A list of tuples, where each tuple contains
            four integers (start_row, end_row, start_col, end_col) specifying the
            indices of the block to extract.

    Returns:
        numpy.ndarray: The resulting matrix containing the extracted blocks.
    """
    result_blocks = []

    for indices in block_indices:
        start_row, end_row, start_col, end_col = indices
        block = matrix[start_row:end_row + 1, start_col:end_col + 1]
        result_blocks.append(block)

    # Combine the extracted blocks into a new matrix
    result_matrix = np.block(result_blocks)

    return result_matrix

def slice_matrix(matrix, bounds):
    """
    Slice a matrix based on the given 4-tuple bounds.

    Args:
    matrix (list of lists): The input matrix.
    bounds (tuple): A 4-tuple (start_row, end_row, start_col, end_col) specifying the slice bounds.

    Returns:
    list of lists: The sliced submatrix.
    """
    start_row, end_row, start_col, end_col = bounds
    sliced_matrix = np.array([row[start_col:end_col] for row in matrix[start_row:end_row]])
    return sliced_matrix

In [ ]:
rebin_factor = 60 ## [40,60,80,100]
energy = 750 ## [500,625,750,875,1000]
width = 70 ## [70,100,130]
energy_width = width
flux_alpha = 1e-12

solutions_sys = np.load("UnfoldingResults/GaussianSmearingSystematicUnfoldingEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy")
solutions_stat = np.load("UnfoldingResults/GaussianSmearingStatisticalUnfoldingEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy")

true = np.load("UnfoldingResults/GaussianSmearingTrueEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy")

modes = ['CCQE','RES','2p2h','Other']
modes_arrays = [np.load("UnfoldingResults/UnfoldedTrueEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Mode"+modes[i]+"Alpha"+str(flux_alpha)+".npy") for i in range(len(modes))]

energy_bin_centers = rebinned_bin_centers(16000,-8000,8000,rebin_factor)/1000

get_virtual_flux(energy,width)

figure(figsize=(8, 6), dpi=120)
sns.set_palette("colorblind")
plt.plot(energy_bin_centers[16000//(rebin_factor*2):],solutions_sys.mean(axis=1),label='Virtual flux reco')
plt.plot(energy_bin_centers[16000//(rebin_factor*2):],true[len(true)//2:],label='Virtual flux true')

for i in range(len(modes)):
    plt.plot(energy_bin_centers[16000//(rebin_factor*2):],modes_arrays[i][len(true)//2:],label=modes[i])
plt.xlabel(r'$\omega$ [GeV]')
plt.ylabel(r'$\frac{d\sigma}{d\omega}$ [$10^{-38} \frac{cm^{2}}{GeV}$]')
plt.grid(True, color = "grey", linewidth = "1", linestyle = "-")
plt.xlim([0,1.4])
plt.legend()

##By the end of this block, data is loaded and energy_bin_centers is ready

In [ ]:
## Prepare data and uncertainties
cv = solutions_sys.mean(axis=1)
centers = energy_bin_centers[len(energy_bin_centers)//2:]

## Systematics
solutions_sys_t = solutions_sys.T
row_sums = np.sum(solutions_sys_t, axis=1)
expanded_sums = np.expand_dims(row_sums, axis=1)
solutions_sys_normed = solutions_sys_t / expanded_sums
solutions_sys_normed[:,-1] = np.array(row_sums)

cov_test = np.cov(solutions_sys_normed.T)
sums_mean = solutions_sys.T.sum(axis=1).mean()
shape = np.sqrt(cov_test.diagonal()[:-1])/(solutions_sys_normed.T[:-1,:].mean(axis=1))
shape_unc = np.abs(cv*np.append(shape,0))
norm_unc = cv*np.sqrt(cov_test[-1,-1])/sums_mean

## Statistics
stat_unc = np.sqrt(np.diagonal(np.cov(solutions_stat)))

## Stat + Shape
stat_shape_unc = np.sqrt(stat_unc**2+shape_unc**2)

## By the end of this block, cv = reco; centers = x axis; shape_unc and norm_unc are the decomposed uncertainties

In [ ]:
plt.rcParams['font.size'] = 22
plt.rcParams['text.usetex'] = True
modes_bool = True

if modes_bool == True:
    modes_lst = [mode for mode in modes]

unfolded = True
residuals = -(np.array(true[len(true)//2:]) - np.array(cv))

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(12, 8), gridspec_kw={'height_ratios': [8, 1]}, dpi=300)

col = 'mediumvioletred'

ax1.errorbar(centers,cv,yerr=stat_unc,color=col ,fmt='.', linewidth=0.5, capsize=2, capthick=1, ecolor=col)
ax1.set_xlim(0.0,1.0)
ax1.set_ylabel(r'$\frac{d\sigma}{d\omega}$ [$10^{-38} \frac{cm^{2}}{GeV}$]',fontsize=24)
if unfolded == True:
    unfolded_str = r'Unfolded '
else:
    unfolded_str = ''
legend_label = 'Unfolded \n Stat $\\bigoplus$ Sys Shape Unc'
ax1.errorbar(centers,cv,yerr=stat_shape_unc,color=col,label=legend_label,fmt='.', linewidth=0.5, capsize=2, capthick=1,ecolor=col)
ax1.plot(centers,true[len(true)//2:],color='black',label='True (Virtual $\\sigma(E_{\\nu}-E_{\mu})$)')

if modes_bool == True:
    for i in range(len(modes)):
        ax1.plot(energy_bin_centers,modes_arrays[i],label=modes[i],linestyle='--')
        
ax1.fill_between(centers,0,norm_unc,color='lightsteelblue',label=r"Norm Unc")
ax1.legend(fontsize=17)

# Plot the residuals
ax2.errorbar(centers, residuals, yerr=stat_shape_unc, color=col, fmt='.', linewidth=0.5, capsize=2, capthick=1,ecolor=col)
ax2.errorbar(centers, residuals, yerr=stat_unc, color=col, fmt='.', linewidth=0.5, capsize=2, capthick=1,ecolor=col)
ax2.axhline(y=0, color='black')
ax2.set_xlabel(r'$\omega$ $[GeV]$',fontsize=24)

ax1.grid(True, color = "gainsboro", linewidth = "1", linestyle = "-")
ax2.grid(True, color = "gainsboro", linewidth = "1", linestyle = "-")
fmt = ticker.FormatStrFormatter('%0.2f')
ax1.yaxis.set_major_formatter(fmt)
ax1.xaxis.set_major_formatter(fmt)
ax2.yaxis.set_major_formatter(fmt)

plt.subplots_adjust(hspace=0.05)

modified = False

ax1.set_title('Unfolded result for a $\\tilde{E}_{\\nu} = '+str(1e-3*energy)+'$ $GeV$, $\sigma$ $\\approx '+str(1e-3*width)+'$ $GeV$ virtual flux (10 years of data)',fontsize=20)
if unfolded == False:
    ax1.set_title('$\mu = '+str(1e-3*energy)+'$ $GeV$, $\sigma$ $\\approx '+str(1e-3*width)+'$ $GeV$ virtual flux (20 years of data)')
    ax2.set_xlabel('$\omega$ [$GeV$]')
plt.show()